<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br>
<b>Investigate sending ApPipe metrics to Sasquatch</b> <br>
Contact author: Ian Sullivan<br>
Last verified to run: 04 January 2024<br>
LSST Science Piplines version: d_2024_03_07<br>
Run at USDF on the "2021 Diffim Sprint" HSC dataset


## Run details

Baseline run in comparison with pre-convolution

* Used daily `w_2024_08` with `ip_diffim`, `analysis_tools`, `ap_pipe`, `ap_association` set up locally 
* rbClassifier_data set up locally
* Run at USDF:  /sdf/data/rubin/user/sullii/tickets/DM-41875/submit/u/sullii/DM-41875/HSC/main/20240131T191654Z and /sdf/data/rubin/user/sullii/tickets/DM-41875/submit/u/sullii/DM-41875/HSC/centerAll/20240201T201645Z



### 1. Main package imports

In [1]:
import os
import importlib
import pprint

In [2]:
import matplotlib.pyplot as plt
#%matplotlib widget
%matplotlib inline

import numpy as np
import pandas as pd

In [3]:
import astropy.units as u
from astropy.coordinates import SkyCoord

In [4]:
import lsst.afw.display as afwDisplay
import lsst.geom

import lsst.daf.butler as dafButler
import lsst.pipe.base

In [5]:
!eups list analysis_ap

   LOCAL:/home/s/sullii/project/build/analysis_ap 	setup


In [6]:
from lsst.analysis.ap import apdb

In [7]:
from lsst.analysis.ap import legacyApdbUtils as utils
from lsst.analysis.ap import legacyPlotUtils as plac
from lsst.analysis.ap import nb_utils

In [8]:
from astropy.table import Table

In [9]:
importlib.reload(plac)

<module 'lsst.analysis.ap.legacyPlotUtils' from '/home/s/sullii/project/build/analysis_ap/python/lsst/analysis/ap/legacyPlotUtils.py'>

In [10]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

importlib.reload(utils)

<module 'lsst.analysis.ap.legacyApdbUtils' from '/home/s/sullii/project/build/analysis_ap/python/lsst/analysis/ap/legacyApdbUtils.py'>

In [11]:
from lsst_efd_client import EfdClient
client = EfdClient("usdfdev_efd", db_name="lsst.dm")

In [12]:
await client.get_topics()

['lsst.dm.ap12PsfSky',
 'lsst.dm.astromDiffMetrics',
 'lsst.dm.blendMetrics',
 'lsst.dm.calexpMetrics',
 'lsst.dm.calibrate',
 'lsst.dm.constantTool',
 'lsst.dm.diaSourcesGoodVsBadRatio',
 'lsst.dm.diffimMetadataMetric',
 'lsst.dm.e1Diff',
 'lsst.dm.e1DiffScatterMetric',
 'lsst.dm.e2Diff',
 'lsst.dm.e2DiffScatterMetric',
 'lsst.dm.isolatedDeblenderMetrics',
 'lsst.dm.matchedRefCModelFluxChiMetric',
 'lsst.dm.matchedRefCModelMagChi',
 'lsst.dm.matchedRefCModelMagDiff',
 'lsst.dm.matchedRefCModelMagDiffMetric',
 'lsst.dm.matchedRefMagChiMetric',
 'lsst.dm.matchedRefPositionXChi',
 'lsst.dm.matchedRefPositionXChiMetric',
 'lsst.dm.matchedRefPositionXDiff',
 'lsst.dm.matchedRefPositionXDiffMetric',
 'lsst.dm.matchedRefPositionYChi',
 'lsst.dm.matchedRefPositionYChiMetric',
 'lsst.dm.matchedRefPositionYDiff',
 'lsst.dm.matchedRefPositionYDiffMetric',
 'lsst.dm.numDiaSourcesAll',
 'lsst.dm.numDiaSourcesHighReliability',
 'lsst.dm.numDiaSourcesLowReliability',
 'lsst.dm.numDiaSourcesNanReliab

In [13]:
query = '''SELECT * FROM "lsst.dm.numDiaSourcesHighReliability" '''
df = await client.influx_client.query(query)
df

,band,band_1,dataset_tag,dataset_tag_1,dataset_type,detector,detector_1,exposure,id,instrument,...,reference_package_timestamp,reference_package_version,run,run_1,run_timestamp,skymap,timestamp,tract,visit,visit_1
2024-03-07 02:12:03+00:00,g,g,Generic,Generic,assocDiaSrcCore_metrics,49,49,,10316fc7-1bd4-493e-ba86-9347134316f2,HSC,...,1709193600,g4213664e8e+38e8488126,u/sullii/DM-41108/HSC/ticket/20240307T021203Z,u/sullii/DM-41108/HSC/ticket/20240307T021203Z,1709777523,,1709777523,,11694,11694
2024-03-07 02:12:03+00:00,g,g,Generic,Generic,assocDiaSrcCore_metrics,49,49,,461dbdeb-613d-4a1b-8cae-2741c182d38b,HSC,...,1709193600,g4213664e8e+38e8488126,u/sullii/DM-41108/HSC/ticket/20240307T021203Z,u/sullii/DM-41108/HSC/ticket/20240307T021203Z,1709777523,,1709777523,,11710,11710
2024-03-07 02:12:03+00:00,g,g,Generic,Generic,assocDiaSrcCore_metrics,49,49,,4bd72c74-b52e-4f1b-a47c-84365f065958,HSC,...,1709193600,g4213664e8e+38e8488126,u/sullii/DM-41108/HSC/ticket/20240307T021203Z,u/sullii/DM-41108/HSC/ticket/20240307T021203Z,1709777523,,1709777523,,11700,11700
2024-03-07 02:12:03+00:00,g,g,Generic,Generic,assocDiaSrcCore_metrics,49,49,,59a9402b-bd97-4eaf-a179-82d8292630cc,HSC,...,1709193600,g4213664e8e+38e8488126,u/sullii/DM-41108/HSC/ticket/20240307T021203Z,u/sullii/DM-41108/HSC/ticket/20240307T021203Z,1709777523,,1709777523,,11706,11706
2024-03-07 02:12:03+00:00,g,g,Generic,Generic,assocDiaSrcCore_metrics,49,49,,68e4a2e1-e329-4fbf-bcbc-5259824eef97,HSC,...,1709193600,g4213664e8e+38e8488126,u/sullii/DM-41108/HSC/ticket/20240307T021203Z,u/sullii/DM-41108/HSC/ticket/20240307T021203Z,1709777523,,1709777523,,11690,11690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-08 01:40:42+00:00,g,g,Generic,Generic,assocDiaSrcCore_metrics,66,66,,79d95225-4d8d-40f6-810a-10511762cec4,HSC,...,1709798400,g4213664e8e+399544b0f1,u/sullii/DM-41108/HSC/ticket/20240308T014042Z,u/sullii/DM-41108/HSC/ticket/20240308T014042Z,1709862042,,1709862042,,29340,29340
2024-03-08 01:40:42+00:00,g,g,Generic,Generic,assocDiaSrcCore_metrics,66,66,,83f0b39f-82fa-4205-8297-c84b41ea812d,HSC,...,1709798400,g4213664e8e+399544b0f1,u/sullii/DM-41108/HSC/ticket/20240308T014042Z,u/sullii/DM-41108/HSC/ticket/20240308T014042Z,1709862042,,1709862042,,11706,11706
2024-03-08 01:40:42+00:00,g,g,Generic,Generic,assocDiaSrcCore_metrics,66,66,,a13cd54d-0837-4fc8-b956-06e88b3f8a00,HSC,...,1709798400,g4213664e8e+399544b0f1,u/sullii/DM-41108/HSC/ticket/20240308T014042Z,u/sullii/DM-41108/HSC/ticket/20240308T014042Z,1709862042,,1709862042,,11690,11690
2024-03-08 01:40:42+00:00,g,g,Generic,Generic,assocDiaSrcCore_metrics,66,66,,ad375bef-0488-4612-889d-aaa000d7f3d0,HSC,...,1709798400,g4213664e8e+399544b0f1,u/sullii/DM-41108/HSC/ticket/20240308T014042Z,u/sullii/DM-41108/HSC/ticket/20240308T014042Z,1709862042,,1709862042,,11692,11692


In [14]:
df.columns

Index(['band', 'band_1', 'dataset_tag', 'dataset_tag_1', 'dataset_type',
       'detector', 'detector_1', 'exposure', 'id', 'instrument',
       'instrument_1', 'numDiaSourcesHighReliability', 'patch',
       'physical_filter', 'physical_filter_1', 'reference_package',
       'reference_package_timestamp', 'reference_package_version', 'run',
       'run_1', 'run_timestamp', 'skymap', 'timestamp', 'tract', 'visit',
       'visit_1'],
      dtype='object')

In [15]:
df['numDiaSourcesHighReliability']

2024-03-07 02:12:03+00:00    0
2024-03-07 02:12:03+00:00    8
2024-03-07 02:12:03+00:00    0
2024-03-07 02:12:03+00:00    2
2024-03-07 02:12:03+00:00    4
                            ..
2024-03-08 01:40:42+00:00    7
2024-03-08 01:40:42+00:00    6
2024-03-08 01:40:42+00:00    6
2024-03-08 01:40:42+00:00    3
2024-03-08 01:40:42+00:00    7
Name: numDiaSourcesHighReliability, Length: 144, dtype: int64

In [22]:
query = '''SELECT * FROM "lsst.dm.diffimMetadataMetric" '''
df = await client.influx_client.query(query)
df

,band,band_1,dataset_tag,dataset_tag_1,dataset_type,detector,detector_1,exposure,id,instrument,...,scaleScienceVarianceFactor,scaleTemplateVarianceFactor,sciencePsfSize,skymap,templateCoveragePercent,templatePsfSize,timestamp,tract,visit,visit_1
2024-03-07 02:12:03+00:00,g,g,Generic,Generic,diffimMetadata_metrics,49,49,,0a3673ce-95ed-4ba7-89b0-84f1ee99c5fb,HSC,...,1.148993,1.239419,4.025803,,NaN,3.907133,1709777523,,11692,11692
2024-03-07 02:12:03+00:00,g,g,Generic,Generic,diffimMetadata_metrics,49,49,,1a1931a7-3ec6-4fd7-9c51-00101104ab0c,HSC,...,1.148467,1.237540,3.994159,,NaN,3.928339,1709777523,,11696,11696
2024-03-07 02:12:03+00:00,g,g,Generic,Generic,diffimMetadata_metrics,49,49,,2c621b0a-bacc-4014-9c2b-65de0b4be2a1,HSC,...,1.148715,1.235386,4.256901,,NaN,3.897487,1709777523,,11708,11708
2024-03-07 02:12:03+00:00,g,g,Generic,Generic,diffimMetadata_metrics,49,49,,39bdb804-0eae-40b5-8000-aaf3f264cc55,HSC,...,1.149416,1.232716,4.239019,,NaN,3.899275,1709777523,,11690,11690
2024-03-07 02:12:03+00:00,g,g,Generic,Generic,diffimMetadata_metrics,49,49,,488340b4-87ff-4892-8cbd-5edf49238fb8,HSC,...,1.149179,1.235854,4.092661,,NaN,3.897836,1709777523,,11700,11700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-08 01:40:42+00:00,g,g,Generic,Generic,diffimMetadata_metrics,66,66,,5986957e-9505-4045-9c17-02687f20edd1,HSC,...,1.137714,1.226445,3.932159,,99.576447,3.920856,1709862042,,11694,11694
2024-03-08 01:40:42+00:00,g,g,Generic,Generic,diffimMetadata_metrics,66,66,,843abd09-a19c-46b9-9663-00b663ee2c99,HSC,...,1.139850,1.231111,3.725009,,99.541748,3.892366,1709862042,,11704,11704
2024-03-08 01:40:42+00:00,g,g,Generic,Generic,diffimMetadata_metrics,66,66,,a3bc3350-41ab-4729-9109-b3b3e995e56c,HSC,...,1.138051,1.201797,4.767076,,99.719063,3.924847,1709862042,,11710,11710
2024-03-08 01:40:42+00:00,g,g,Generic,Generic,diffimMetadata_metrics,66,66,,c2c267f5-bf4f-4ee9-ba7f-7ad9a5a41656,HSC,...,1.138270,1.220455,4.110731,,99.600174,3.913900,1709862042,,11706,11706


In [25]:
query = '''SELECT * FROM "lsst.dm.numDipoles" '''
df = await client.influx_client.query(query)
df['numDipoles']

2024-03-07 02:12:03+00:00    25
2024-03-07 02:12:03+00:00    55
2024-03-07 02:12:03+00:00    32
2024-03-07 02:12:03+00:00    24
2024-03-07 02:12:03+00:00    36
                             ..
2024-03-08 01:40:42+00:00    26
2024-03-08 01:40:42+00:00    38
2024-03-08 01:40:42+00:00    41
2024-03-08 01:40:42+00:00    32
2024-03-08 01:40:42+00:00    28
Name: numDipoles, Length: 144, dtype: int64

In [29]:

query = '''SELECT * FROM "lsst.dm.diaSourcesGoodVsBadRatio" '''
df = await client.influx_client.query(query)
df['DiaSourcesGoodVsBadRatio']

2024-03-07 02:12:03+00:00    0.051948
2024-03-07 02:12:03+00:00    0.028986
2024-03-07 02:12:03+00:00    0.033333
2024-03-07 02:12:03+00:00    0.056338
2024-03-07 02:12:03+00:00    0.014085
                               ...   
2024-03-08 01:40:42+00:00    0.052632
2024-03-08 01:40:42+00:00    0.127273
2024-03-08 01:40:42+00:00    0.052632
2024-03-08 01:40:42+00:00    0.090909
2024-03-08 01:40:42+00:00    0.118012
Name: DiaSourcesGoodVsBadRatio, Length: 144, dtype: float64

# Conclusion

This demonstrates that the ApPipe metrics written with the analysis_tools afterburner can be read from the EFD, which means that they were sucessfully uploaded through Sasquatch